In [2]:
import pandas as pd
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [4]:
features = [x for x in train.columns if x.startswith("var")]


In [6]:
hist_df = pd.DataFrame()
for var in features:
    var_stats = train[var].append(test[var]).value_counts()
    hist_df[var] = pd.Series(test[var]).map(var_stats)
    hist_df[var] = hist_df[var] > 1
    
ind = hist_df.sum(axis=1) != 200
var_stats = {var:train[var].append(test[ind][var]).value_counts() for var in features}

In [7]:
import numpy as np
#import pandas as pd
import lightgbm as lgb
from scipy.special import logit
pred = 0
for var in features:
    model = lgb.LGBMClassifier(**{
        'learning_rate':0.05, 'max_bin': 165, 'max_depth': 5, 'min_child_samples': 150,
        'min_child_weight': 0.1, 'min_split_gain': 0.0018, 'n_estimators': 41,
        'num_leaves': 6, 'reg_alpha': 2.0, 'reg_lambda': 2.54, 'objective': 'binary', 'n_jobs': -1})
    model = model.fit(np.hstack([train[var].values.reshape(-1,1),
                                 train[var].map(var_stats[var]).values.reshape(-1,1)]),
                               train["target"].values)
    pred += logit(model.predict_proba(np.hstack([test[var].values.reshape(-1,1),
                                 test[var].map(var_stats[var]).values.reshape(-1,1)]))[:,1])

In [9]:
pd.DataFrame({"ID_code":test["ID_code"], "target":pred}).to_csv("submission.csv", index=False)